
# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [ ]:
# !wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
# !tar xf spark-3.0.1-bin-hadoop2.7.tgz
# !pip install -q findspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [1]:
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "../spark-3.0.1-bin-hadoop2.7"

Run a local spark session to test your installation:

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Downloaded data

In [1]:
# ! wget http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2020-08-18/data/listings.csv
# ! wget http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2020-08-18/data/reviews.csv
# ! wget http://data.insideairbnb.com/spain/catalonia/barcelona/2020-09-12/visualisations/listings.csv

--2020-11-03 20:42:45--  http://data.insideairbnb.com/spain/catalonia/barcelona/2020-09-12/visualisations/listings.csv
Resolving data.insideairbnb.com (data.insideairbnb.com)... 52.216.77.19
Connecting to data.insideairbnb.com (data.insideairbnb.com)|52.216.77.19|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3137507 (3,0M) [application/csv]
Saving to: ‘listings.csv’

listings.csv        100%[===================>]   2,99M   836KB/s    in 3,7s    

2020-11-03 20:42:50 (836 KB/s) - ‘listings.csv’ saved [3137507/3137507]



In [3]:
import pandas as pd
import tqdm

In [58]:
listings = spark.read.csv('listings.csv',inferSchema=True, header =True)

train = spark.read.parquet('train_test_data/train.pkt')
test = spark.read.parquet('train_test_data/test.pkt')

In [59]:
# listings_df_pd = pd.read_csv("listings.csv")
# listings_df_pd.to_parquet("listing.pkt")
# listings = spark.read.parquet("listing.pkt")
# listings_not_null = listings[listings.name.isNotNull()]

In [60]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark.ml.feature import BucketedRandomProjectionLSH


tokenizer = Tokenizer(inputCol="name", outputCol="CleanTokens")
stopwordsremover = StopWordsRemover(inputCol="CleanTokens", outputCol="CleanTokensStopRemoved")
hashingTF = HashingTF(inputCol="CleanTokensStopRemoved", outputCol="VectorSpace")
idf = IDF(inputCol="VectorSpace", outputCol="VectorSpaceIDF")



In [61]:
pipeline = Pipeline(stages=[tokenizer, stopwordsremover, hashingTF, idf])
pipelineModel = pipeline.fit(train)

In [62]:
df_pipe = pipelineModel.transform(train)

Learning LSH

In [63]:
brp = BucketedRandomProjectionLSH(inputCol="VectorSpaceIDF", outputCol="hashes", bucketLength=20,
                                  numHashTables=10)
model = brp.fit(df_pipe)
df_pipe = model.transform(df_pipe)


In [64]:
pipelineModelTest = pipeline.fit(test)
test_prepared = pipelineModel.transform(test)

In [65]:
test_pd = test_prepared.toPandas()

In [66]:
looking_for = test_pd.VectorSpaceIDF.to_list()

In [67]:
def find_neighbours(data, value, number, colName):
    result = model.approxNearestNeighbors(data, value, number, distCol=colName)
  
    return result.select("id").toPandas()['id'][1:].to_list()



In [68]:
out = []
for key in tqdm.tqdm_notebook(looking_for):
    neigh = find_neighbours(test_prepared, key, 6, 'hashes')
    out.append(neigh)

<ipython-input-68-117033a312df>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for key in tqdm.tqdm_notebook(looking_for):


It's slow anyway

In [69]:
import numpy as np

In [94]:
def compare_lists(a,b):
    results = []
    for i, _ in enumerate(a):
        intersection = set(a[i]).intersection(b[i])
        results.append(len(intersection))
        
    return np.mean(results), np.sum(np.array(results) > 1)/len(b), np.sum(np.array(results) == 5)/len(b) 

In [95]:
score_a, score_b, score_c = compare_lists(out, test_pd['ground_truth'])

In [93]:
score_a, score_b, score_c

(1.7424354243542435, 0.5682656826568265, 0.07060270602706027)

In [39]:
test_pd.loc[test_pd.id.isin(out[5])].name

177           Beautiful Apartment Sagrada Familia
206           Gran Vía Apartment Catalunya square
285            Beautiful Double Room With Balcony
327                                Cozy Apartment
331    Beautiful Double Room with private terrace
Name: name, dtype: object

In [43]:
test_pd['ground_truth'][5]

[10894127, 10874782, 543342, 43482809, 10127485]